## Using AST >> Syntax Comparison

In [7]:
import ast
from sentence_transformers import SentenceTransformer, util

In [8]:
def get_ast_structure(code_str):
    try:
        tree = ast.parse(code_str)
        return ast.dump(tree)
    except SyntaxError:
        return None

In [9]:
code1 = "def add(a, b): return a + b"
code2 = "def sum(x, y): return x + y"

print(get_ast_structure(code1))
print(get_ast_structure(code2))

Module(body=[FunctionDef(name='add', args=arguments(posonlyargs=[], args=[arg(arg='a', annotation=None, type_comment=None), arg(arg='b', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())))], decorator_list=[], returns=None, type_comment=None)], type_ignores=[])
Module(body=[FunctionDef(name='sum', args=arguments(posonlyargs=[], args=[arg(arg='x', annotation=None, type_comment=None), arg(arg='y', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Name(id='y', ctx=Load())))], decorator_list=[], returns=None, type_comment=None)], type_ignores=[])


In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')

vec1 = model.encode(code1, convert_to_tensor=True)
vec2 = model.encode(code2, convert_to_tensor=True)

similarity = util.pytorch_cos_sim(vec1, vec2)
print(f"Similarity Score: {similarity.item()*100:.2f}%")

Similarity Score: 76.04%


In [13]:
code1 = """def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b
"""
code2 = """def F(n):
    if n <= 1:
        return n
    else:
        return F(n - 1) + F(n - 2)
"""

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')

vec1 = model.encode(code1, convert_to_tensor=True)
vec2 = model.encode(code2, convert_to_tensor=True)

similarity = util.pytorch_cos_sim(vec1, vec2)
print(f"Similarity Score: {similarity.item()*100:.2f}%")

Similarity Score: 64.33%


| Similarity % | Meaning                                         | Action              |
| ------------ | ----------------------------------------------- | ------------------- |
| 0–30%        | Likely original                                 | No issue            |
| 30–70%       | Some overlap (could be accidental/common logic) | Review manually     |
| 70–90%   | Likely copied or reused code                    | Investigate closely |
| 90–100%      | Almost identical                                | Strong plagiarism   |


## AST Based >> Logic Comparison

In [15]:
# Re-import and re-run due to kernel reset

import ast
import difflib

# Define the two code samples
code1 = """def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b
"""

code2 = """def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
"""

# Parse into AST and dump structure
def get_ast_structure(code):
    try:
        tree = ast.parse(code)
        return ast.dump(tree, annotate_fields=False, include_attributes=False)
    except SyntaxError:
        return ""

ast1 = get_ast_structure(code1)
ast2 = get_ast_structure(code2)

# Use difflib to measure structure-level similarity
similarity = difflib.SequenceMatcher(None, ast1, ast2).ratio() * 100

similarity


26.877470355731226

In [16]:
code1 = "def add(a, b): return a + b"
code2 = "def sum(x, y): return x + y"

In [17]:
ast1 = get_ast_structure(code1)
ast2 = get_ast_structure(code2)

# Use difflib to measure structure-level similarity
similarity = difflib.SequenceMatcher(None, ast1, ast2).ratio() * 100

similarity

96.41025641025641

In [18]:
code1 = """def fibonacci_iterative(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b
"""
code2 = """def fibonacci_recursive(n):
    if n <= 1:
        return n
    else:
        return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

for i in range(10):
    print(fibonacci_recursive(i), end=" ") # Prints the first 10 Fibonacci numbers
"""

In [19]:
ast1 = get_ast_structure(code1)
ast2 = get_ast_structure(code2)

# Use difflib to measure structure-level similarity
similarity = difflib.SequenceMatcher(None, ast1, ast2).ratio() * 100

similarity

30.59006211180124

## GitHub Code Search

In [1]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# GitHub token (generate from https://github.com/settings/tokens, no scopes needed)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")  # Get the token from environment variable
if not GITHUB_TOKEN:
    raise ValueError("GitHub token not found. Please set the GITHUB_TOKEN environment variable.")

# Search query (can be a code fragment or function name)
query = "return fibonacci(n-1) + fibonacci(n-2)"


# GitHub Search API URL
url = f"https://api.github.com/search/code?q={query}+in:file+language:python"

# Headers with auth token
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# Send request
response = requests.get(url, headers=headers)

# Parse results
if response.status_code == 200:
    results = response.json()
    total = results['total_count']
    print(f"🔍 Found {total} matching code file(s) on GitHub:")
    
    for item in results['items'][:5]:  # Show top 5
        print(f"- {item['name']} at {item['html_url']}")
else:
    print(f"❌ GitHub API error: {response.status_code}")


🔍 Found 1336 matching code file(s) on GitHub:
- recurrsion.py at https://github.com/calistus-igwilo/python/blob/796f65dd6ebf29434481f7ddfce793910c42d8ca/recurrsion.py
- template.py at https://github.com/ArmelRandy/Self-instruct/blob/c58db251b785136e9b54ee3c915426d68d788e78/template.py
- d12_fib.py at https://github.com/19ceng/ceng104pro/blob/ebbb930fec94fc4a8c58b30a228bee312979a409/src/d12_fib.py
- bench.py at https://github.com/moraes/webapp-improved/blob/0e6218dcd3ba2e0ba0c6a6c87ba4fbe1eab287c4/lib/appengine-ndb-experiment/bench.py
- FiBoNaCci.py at https://github.com/technojam/Hacktoberfest-2020-Baby/blob/d1d757d8e23a8ed0b212c2d3a757c23f97eb6c6e/FiBoNaCci.py


In [ ]:
import requests

class PlagiaGuard:
    def __init__(self, code):
        self.code = code
        self.show_results()

    def logic(self):
        GITHUB_TOKEN = "token"

        query = self.code

        url = f"https://api.github.com/search/code?q={query}+in:file+language:python"

        headers = {
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github.v3+json"
        }

        return requests.get(url, headers=headers)

    def output(self):
        response = self.logic()
        if response.status_code == 200:
            return response.json()
        else:
            return False
        

    def show_results(self):
        results = self.output()
        if results:
            links = {}
            for item in results['items'][:10]:  # Show top 5
                links[item['name']] = item['html_url']
            return links
        else:
            return f"GitHub API error: {self.logic().status_code}"


In [21]:
pg = PlagiaGuard("def factorial(n)")

In [22]:
pg.show_results()

{'DEF.py': 'https://github.com/matcarballo/Inform-tica-General/blob/1f89debc6764f379cb3aa736f8da0b459493ce0a/DEF.py',
 'ocl.py': 'https://github.com/mdipierro/ocl/blob/53b8400d27e09e857ebbf146b9dce5a17fb693b8/ocl.py',
 'hola.py': 'https://github.com/reingart/rad2py/blob/e4802ade132d569225e8be7eca830d2f888aa5f9/hola.py',
 'LIA.py': 'https://github.com/adelq/rosalind/blob/2ce22ed52b810f6e31be6d7d65d89c0a89356f29/LIA.py',
 'Q.3.py': 'https://github.com/AunZaidii/practice-codes/blob/1d33ee400a0f03b7a79699b8594fb9263e972ef7/Q.3.py',
 'app.py': 'https://github.com/Advaitgaur004/Auto-Scaling/blob/29036d8f16855378f956f60a69245ea852d645f8/app.py',
 '5-5.py': 'https://github.com/aaronlab/python-algorithm/blob/3046ea460a3f29224fba1ea03b83fed903c5c76b/5-5.py',
 '3.py': 'https://github.com/ArthurJraghatspanyan/ArtJR2003-Python_Tasks-16/blob/3d3ea0296b1defca319a704338e0afe6b02af7e2/3.py',
 'fact.py': 'https://github.com/SanaSNavas7/Python/blob/df8ef4d34cbfc544606b445ece52ead25093fd92/fact.py',
 'fac

In [23]:
pg.show_results().keys(), pg.show_results().values()

(dict_keys(['DEF.py', 'ocl.py', 'hola.py', 'LIA.py', 'Q.3.py', 'app.py', '5-5.py', '3.py', 'fact.py', 'fac.py']),
 dict_values(['https://github.com/matcarballo/Inform-tica-General/blob/1f89debc6764f379cb3aa736f8da0b459493ce0a/DEF.py', 'https://github.com/mdipierro/ocl/blob/53b8400d27e09e857ebbf146b9dce5a17fb693b8/ocl.py', 'https://github.com/reingart/rad2py/blob/e4802ade132d569225e8be7eca830d2f888aa5f9/hola.py', 'https://github.com/adelq/rosalind/blob/2ce22ed52b810f6e31be6d7d65d89c0a89356f29/LIA.py', 'https://github.com/AunZaidii/practice-codes/blob/1d33ee400a0f03b7a79699b8594fb9263e972ef7/Q.3.py', 'https://github.com/Advaitgaur004/Auto-Scaling/blob/29036d8f16855378f956f60a69245ea852d645f8/app.py', 'https://github.com/aaronlab/python-algorithm/blob/3046ea460a3f29224fba1ea03b83fed903c5c76b/5-5.py', 'https://github.com/ArthurJraghatspanyan/ArtJR2003-Python_Tasks-16/blob/3d3ea0296b1defca319a704338e0afe6b02af7e2/3.py', 'https://github.com/SanaSNavas7/Python/blob/df8ef4d34cbfc544606b445ece5

In [24]:
for name, url in pg.show_results().items():
    print(f"{name} -- {url}")

DEF.py -- https://github.com/matcarballo/Inform-tica-General/blob/1f89debc6764f379cb3aa736f8da0b459493ce0a/DEF.py
ocl.py -- https://github.com/mdipierro/ocl/blob/53b8400d27e09e857ebbf146b9dce5a17fb693b8/ocl.py
hola.py -- https://github.com/reingart/rad2py/blob/e4802ade132d569225e8be7eca830d2f888aa5f9/hola.py
LIA.py -- https://github.com/adelq/rosalind/blob/2ce22ed52b810f6e31be6d7d65d89c0a89356f29/LIA.py
Q.3.py -- https://github.com/AunZaidii/practice-codes/blob/1d33ee400a0f03b7a79699b8594fb9263e972ef7/Q.3.py
app.py -- https://github.com/Advaitgaur004/Auto-Scaling/blob/29036d8f16855378f956f60a69245ea852d645f8/app.py
5-5.py -- https://github.com/aaronlab/python-algorithm/blob/3046ea460a3f29224fba1ea03b83fed903c5c76b/5-5.py
3.py -- https://github.com/ArthurJraghatspanyan/ArtJR2003-Python_Tasks-16/blob/3d3ea0296b1defca319a704338e0afe6b02af7e2/3.py
fact.py -- https://github.com/SanaSNavas7/Python/blob/df8ef4d34cbfc544606b445ece52ead25093fd92/fact.py
fac.py -- https://github.com/jueunlee0529

In [25]:
for i, url in enumerate(pg.show_results().values()):
    print(f"{i} -- {url}")

0 -- https://github.com/matcarballo/Inform-tica-General/blob/1f89debc6764f379cb3aa736f8da0b459493ce0a/DEF.py
1 -- https://github.com/mdipierro/ocl/blob/53b8400d27e09e857ebbf146b9dce5a17fb693b8/ocl.py
2 -- https://github.com/reingart/rad2py/blob/e4802ade132d569225e8be7eca830d2f888aa5f9/hola.py
3 -- https://github.com/adelq/rosalind/blob/2ce22ed52b810f6e31be6d7d65d89c0a89356f29/LIA.py
4 -- https://github.com/AunZaidii/practice-codes/blob/1d33ee400a0f03b7a79699b8594fb9263e972ef7/Q.3.py
5 -- https://github.com/Advaitgaur004/Auto-Scaling/blob/29036d8f16855378f956f60a69245ea852d645f8/app.py
6 -- https://github.com/aaronlab/python-algorithm/blob/3046ea460a3f29224fba1ea03b83fed903c5c76b/5-5.py
7 -- https://github.com/ArthurJraghatspanyan/ArtJR2003-Python_Tasks-16/blob/3d3ea0296b1defca319a704338e0afe6b02af7e2/3.py
8 -- https://github.com/SanaSNavas7/Python/blob/df8ef4d34cbfc544606b445ece52ead25093fd92/fact.py
9 -- https://github.com/jueunlee0529/Python/blob/2906d9c46d97623170ba38890baeb62de2d8d

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    'Provider Name': ['Iacob Geaorgescu', 'Julius Neumann', 'Christoph Koller', 'Bram Lemmens'],
    'E-mail': ['e-mail@test-email.com'] * 4
}
df = pd.DataFrame(data)
indexes = [i for i in range(1, 6)]
df.set_index(indexes)

# Convert DataFrame to HTML table string
html_table = df.to_html(index=False)

# Print or save the HTML string
print(html_table)

# Optionally, save to an HTML file
# with open('table.html', 'w') as f:
#     f.write(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Provider Name</th>
      <th>E-mail</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>gyyjfhjfhg</td>
      <td>e-mail@test-email.com</td>
    </tr>
    <tr>
      <td>Julius Neumann</td>
      <td>e-mail@test-email.com</td>
    </tr>
    <tr>
      <td>Christoph Koller</td>
      <td>e-mail@test-email.com</td>
    </tr>
    <tr>
      <td>Bram Lemmens</td>
      <td>e-mail@test-email.com</td>
    </tr>
  </tbody>
</table>


In [9]:
indexes = [i for i in range(1, 5)]
indexes

[1, 2, 3, 4]

In [8]:
df

,Provider Name,E-mail
0,Iacob Geaorgescu,e-mail@test-email.com
1,Julius Neumann,e-mail@test-email.com
2,Christoph Koller,e-mail@test-email.com
3,Bram Lemmens,e-mail@test-email.com


In [11]:
df.index = indexes
df

,Provider Name,E-mail
1,Iacob Geaorgescu,e-mail@test-email.com
2,Julius Neumann,e-mail@test-email.com
3,Christoph Koller,e-mail@test-email.com
4,Bram Lemmens,e-mail@test-email.com


In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

class PlagiaGuard:
    def __init__(self, code):
        self.code = code

    def logic(self):
        GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")  # Get the token from environment variable
        if not GITHUB_TOKEN:
            raise ValueError("GitHub token not found. Please set the GITHUB_TOKEN environment variable.")
        
        query = self.code

        url = f"https://api.github.com/search/code?q={query}+in:file+language:python"

        headers = {
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github.v3+json"
        }

        return requests.get(url, headers=headers)

    def output(self):
        response = self.logic()
        if response.status_code == 200:
            return response.json()
        else:
            return {}
        

    def show_results(self):
        results = self.output()
        if results:
            links = {}
            for item in results['items'][:5]:
                links[item['name']] = item['html_url']
            return links
        else:
            return {}
    
    def table_output(self):
        links = self.show_results()
        if links:
            data = {
                "Name" : list(links.keys()),
                "URL" : list(links.values())
            }
            df = pd.DataFrame(data = data)
            indexes = [i for i in range(1, 6)]
            df.index = indexes

            # Make URL clickable by converting to an HTML anchor tag
            df['URL'] = df['URL'].apply(lambda url: f'<a href="{url}" target="_blank" rel="noopener noreferrer">{url}</a>')
            # Convert to HTML
            html_table = df.to_html(escape=False, index=False)

            return html_table
        else:
            return {}

In [54]:
pg = PlagiaGuard("sum = a + b")

In [43]:
pg.show_results()

{'vib': 'https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib',
 'kth-largest-sum-in-a-binary-tree.py': 'https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth-largest-sum-in-a-binary-tree/kth-largest-sum-in-a-binary-tree.py',
 'sum': 'https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum',
 '9.py': 'https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py',
 'b': 'https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b'}

In [44]:
pg.output()

{'total_count': 6213632,
 'incomplete_results': False,
 'items': [{'name': 'vib',
   'path': 'vib',
   'sha': '1db051067a9f144b4aa8547a18b5445dd4b4ce3b',
   'url': 'https://api.github.com/repositories/366314162/contents/vib?ref=188284cdf2864cf5d3969d401f56dbd734483854',
   'git_url': 'https://api.github.com/repositories/366314162/git/blobs/1db051067a9f144b4aa8547a18b5445dd4b4ce3b',
   'html_url': 'https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib',
   'repository': {'id': 366314162,
    'node_id': 'MDEwOlJlcG9zaXRvcnkzNjYzMTQxNjI=',
    'name': 'ViB',
    'full_name': 'max-p-log-p/ViB',
    'private': False,
    'owner': {'login': 'max-p-log-p',
     'id': 52756433,
     'node_id': 'MDQ6VXNlcjUyNzU2NDMz',
     'avatar_url': 'https://avatars.githubusercontent.com/u/52756433?v=4',
     'gravatar_id': '',
     'url': 'https://api.github.com/users/max-p-log-p',
     'html_url': 'https://github.com/max-p-log-p',
     'followers_url': 'https://api.github.co

In [45]:
pg.logic()

<Response [200]>

In [55]:
pg.table_output()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Name</th>\n      <th>URL</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>vib</td>\n      <td>https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib</td>\n    </tr>\n    <tr>\n      <td>kth-largest-sum-in-a-binary-tree.py</td>\n      <td>https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth-largest-sum-in-a-binary-tree/kth-largest-sum-in-a-binary-tree.py</td>\n    </tr>\n    <tr>\n      <td>sum</td>\n      <td>https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum</td>\n    </tr>\n    <tr>\n      <td>9.py</td>\n      <td>https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py</td>\n    </tr>\n    <tr>\n      <td>b</td>\n      <td>https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b</td>\n    </tr

In [29]:
data = {'vib': 'https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib',
 'kth-largest-sum-in-a-binary-tree.py': 'https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth',
 'sum': 'https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum',
 '9.py': 'https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py',
 'b': 'https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b'
}
data = {
    "Name" : list(data.keys()),
    "URL" : list(data.values())
}
df = pd.DataFrame(data = data)
indexes = [i for i in range(1, 6)]
df.index = indexes
df

,Name,URL
1,vib,https://github.com/max-p-log-p/ViB/blob/188284...
2,kth-largest-sum-in-a-binary-tree.py,https://github.com/ChiefRavinder/LeetCode-Subm...
3,sum,https://github.com/mjpost/bin/blob/4b53d26f88d...
4,9.py,https://github.com/20Koen02/AdventOfCode/blob/...
5,b,https://github.com/NUbots/NUClearPortOld/blob/...


In [50]:
data = {'vib': 'https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib',
 'kth-largest-sum-in-a-binary-tree.py': 'https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth-largest-sum-in-a-binary-tree/kth-largest-sum-in-a-binary-tree.py',
 'sum': 'https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum',
 '9.py': 'https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py',
 'b': 'https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b'}

In [52]:
df = pd.DataFrame(data = {
    "Col1" : list(data.keys()),
    "Col2" : list(data.values())
})
indexes = [i for i in range(1, 6)]
df.index = indexes

html_table = df.to_html(index = False)
# html_table = HTML(html_table)
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Col1</th>
      <th>Col2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>vib</td>
      <td>https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib</td>
    </tr>
    <tr>
      <td>kth-largest-sum-in-a-binary-tree.py</td>
      <td>https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth-largest-sum-in-a-binary-tree/kth-largest-sum-in-a-binary-tree.py</td>
    </tr>
    <tr>
      <td>sum</td>
      <td>https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum</td>
    </tr>
    <tr>
      <td>9.py</td>
      <td>https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py</td>
    </tr>
    <tr>
      <td>b</td>
      <td>https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b</td>
    </tr>
  </tbody>
</table>


In [25]:
data.values()

dict_values(['https://github.com/max-p-log-p/ViB/blob/188284cdf2864cf5d3969d401f56dbd734483854/vib', 'https://github.com/ChiefRavinder/LeetCode-Submissions/blob/0d4f4930bf29b2dbf775c98d8a204dcdebfe6355/2646-kth-largest-sum-in-a-binary-tree/kth-largest-sum-in-a-binary-tree.py', 'https://github.com/mjpost/bin/blob/4b53d26f88d5aa901c062dfd9c0f32a6609173cd/sum', 'https://github.com/20Koen02/AdventOfCode/blob/63a8310e84729042440ee118413a7994d33ce535/2019/9.py', 'https://github.com/NUbots/NUClearPortOld/blob/8270ea2326fb47e378c9dd8f9edd68dcf20128fc/b'])